In [18]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, PatternFill


def extract_level(course_code):
    """Extract the level from the course code."""
    if pd.isna(course_code):
        return None
    if "/" in course_code:  # For combined courses
        course_code = course_code.split("/")[0]
    digits = ''.join(filter(str.isdigit, course_code))
    if digits and len(digits) >= 3:
        return int(digits[0])  # The first digit indicates the level
    return None


def process_courses(sheet_path):
    """Process the 'Second Semester' sheet to extract and organize course data."""
    # Known department and college mapping
    department_college_map = {
        "SOC": ("Sociology", "College of Social Sciences"),
        "ECO": ("Economics", "College of Business and Economics"),
        "BIO": ("Biology", "College of Natural Sciences"),
        "BCH": ("Biochemistry", "College of Natural Sciences"),
        "POL": ("Political Science", "College of Social Sciences"),
        "COS": ("Computer Science", "College of Computing"),
        "MAT": ("Mathematics", "College of Natural Sciences"),
        "PHY": ("Physics", "College of Natural Sciences"),
        # Add other mappings as needed
    }

    df = pd.read_excel(sheet_path, sheet_name="Second Semester")
    courses = []

    for _, row in df.iterrows():
        course_code = row["COURSE"]
        if pd.isna(course_code):  # Skip rows with missing COURSE values
            continue
        programs = row["PROGRAMMES OFFERING"]
        exam_type = row["TYPE"]
        level = extract_level(course_code)

        # Ensure programs is a string; otherwise, assign an empty string
        if not isinstance(programs, str):
            programs = ""

        # Split programs into a list of individual programs
        program_list = [program.strip() for program in programs.split(",")]

        for program in program_list:
            # Determine department and college
            for code, (dept, college) in department_college_map.items():
                if code in course_code:
                    department, college = dept, college
                    break
            else:
                department, college = "Unknown Department", "Unknown College"

            courses.append({
                "Course Code": course_code,
                "Program": program,
                "Exam Type": exam_type,
                "Level": level,
                "Department": department,
                "College": college
            })

    return pd.DataFrame(courses)


def process_venues(sheet_path):
    """Process the 'Venues' sheet."""
    df = pd.read_excel(sheet_path, sheet_name="Venues")
    return df.rename(columns={"HALL": "Venue", "CODE": "Code", "EXAM CAPACITY": "Capacity"})


def process_staff(sheet_path):
    """Process the 'List of Staff' sheet."""
    df = pd.read_excel(sheet_path, sheet_name="List of Staff")
    return df.rename(columns={"NAME": "Name", "COLLEGE": "College", "DEPARTMENT": "Department", "PHONE NUMBER": "Phone"})


def adjust_column_width_and_style(writer, sheet_name):
    """Adjust column width and apply header style in Excel."""
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

    # Adjust column widths
    for column_cells in worksheet.columns:
        max_length = max((len(str(cell.value)) for cell in column_cells if cell.value), default=0)
        adjusted_width = max_length + 4  # Add some spacing
        worksheet.column_dimensions[column_cells[0].column_letter].width = adjusted_width

    # Apply header style (yellow background, bold text, centered alignment)
    header_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    header_font = Font(bold=True)
    header_alignment = Alignment(horizontal="center", vertical="center")

    for cell in worksheet[1]:  # First row (header)
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment


def main(input_file, output_file):
    # Process data from Excel sheets
    courses_df = process_courses(input_file)
    venues_df = process_venues(input_file)
    staff_df = process_staff(input_file)

    # Write the processed data to a new Excel file
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        courses_df.to_excel(writer, sheet_name="Courses", index=False)
        adjust_column_width_and_style(writer, "Courses")

        venues_df.to_excel(writer, sheet_name="Venues", index=False)
        adjust_column_width_and_style(writer, "Venues")

        staff_df.to_excel(writer, sheet_name="Staff", index=False)
        adjust_column_width_and_style(writer, "Staff")

    print(f"Processed data saved to {output_file}")


if __name__ == "__main__":
    # Input and output files
    input_excel = "Data for Exam timetable.xlsx"  # Replace with your actual file name
    output_excel = "data10.xlsx"

    if os.path.exists(input_excel):
        main(input_excel, output_excel)
    else:
        print(f"Input file '{input_excel}' not found. Please provide the correct file.")


Processed data saved to data10.xlsx
